In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("data/MNIST/", one_hot=True)

In [ ]:
print(len(data.train.images))
print(len(data.test.images))
print(len(data.validation.images))

In [ ]:
data.train.cls = np.argmax(data.train.labels, axis = 1)
data.test.cls = np.argmax(data.test.labels, axis=1)
print(data.train.cls[:10])
print(data.test.cls[:10])

In [ ]:
data.train.labels.shape

In [ ]:
data.train.cls.shape

In [ ]:
image_size = 28
image_shape = (image_size, image_size)
image_flat = image_size*image_size
num_class = 10

In [ ]:
input_channel = 1
layer1_filter_size = 5
layer1_output_channel = 16
layer2_filter_size = 5
layer2_output_channel = 36
fc1_size = 128
fc2_size = 10

In [ ]:
def get_new_weight(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [ ]:
def get_new_bias(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [ ]:
def get_conv_layer(input, input_channel, output_channel, filter_size, max_pooling=True):
    weight = get_new_weight([filter_size, filter_size, input_channel, output_channel])
    bias = get_new_bias([output_channel])
    
    layer = tf.nn.conv2d(input, filter = weight, strides= [1,1,1,1], padding='SAME')
    
    layer = layer + bias
    
    if max_pooling:
        layer = tf.nn.max_pool(layer, strides = [1,2,2,1], ksize=[1,2,2,1], padding = 'SAME')
        
    layer = tf.nn.relu(layer)
    
    return layer, weight

In [ ]:
def get_flat_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

In [ ]:
def get_fc(flat_layer, input_size, output_size, relu=True):

    weight = get_new_weight([input_size, output_size])
    bias = get_new_bias([output_size])
    
    layer = tf.matmul(flat_layer, weight) + bias
    
    if relu:
        layer = tf.nn.relu(layer)
    return layer, weight

In [ ]:
x = tf.placeholder(tf.float32, [None, image_flat])

In [ ]:
y = tf.placeholder(tf.float32, [None, num_class])

In [ ]:
x_image = tf.reshape(x, [-1, image_size, image_size, input_channel])

In [ ]:
y_true_class = tf.argmax(y, axis = 1)

In [ ]:
layer1, weight1 = get_conv_layer(x_image, input_channel, layer1_output_channel, layer1_filter_size, max_pooling=True)

In [ ]:
layer1

In [ ]:
weight1

In [ ]:
layer2, weight2 = get_conv_layer(layer1, layer1_output_channel,layer2_output_channel , layer2_filter_size, max_pooling=True)

In [ ]:
layer2

In [ ]:
weight2

In [ ]:
flat_layer, num_features = get_flat_layer(layer2)

In [ ]:
flat_layer

In [ ]:
num_features

In [ ]:
layer_fc1, weight_fc1 = get_fc(flat_layer,num_features , fc1_size, relu=True)

In [ ]:
layer_fc1

In [ ]:
weight_fc1

In [ ]:
layer_fc2, weight_fc2 = get_fc(layer_fc1, fc1_size, fc2_size, relu=False)

In [ ]:
layer_fc2

In [ ]:
weight_fc2

In [ ]:
y_pred = tf.nn.softmax(layer_fc2)

In [ ]:
y_pred_class = tf.argmax(y_pred, axis =1)

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,labels=y)

In [ ]:
cost = tf.reduce_mean(cross_entropy)

In [ ]:
optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)

In [ ]:
correct_pred= tf.equal(y_pred_class, y_true_class)

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
BATCH_SIZE = 64

In [ ]:
def optimization(training_iterations):
    for iteration in range(training_iterations):
        x_train, y_train = data.train.next_batch(BATCH_SIZE)
        feed_dict_train = {x:x_train, y:y_train}
        sess.run(optimizer, feed_dict = feed_dict_train)

In [ ]:
def get_accuracy():
    feed_dict_test = {x:data.test.images, y:data.test.labels, y_true_class:data.test.cls}
    print("Accuracy is {0}".format(sess.run(accuracy, feed_dict = feed_dict_test)))

In [ ]:
#get_accuracy()

In [ ]:
#optimization(1)

In [ ]:
optimization(1)